In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# Create an SQLAlchemy Engine
server = 'LAPTOP-5J6JKUOU'
database = 'ContosoRetailDW'
# Connection string for SQL Server using Windows Authentication
engine = create_engine(f'mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&auth=NTLM')


# <span style="color: red;"> DimDate </span>

In [2]:
# Select and cleaning `dimDate`
dimDate = "SELECT * FROM DimDate"
dim_date = pd.read_sql(dimDate, con=engine)

pd.set_option('display.max_rows', None)  # None means unlimited rows
pd.set_option('display.max_columns', None)  # None means unlimited columns

In [3]:
dim_date.head()

,Datekey,CalendarYear,CalendarYearLabel,CalendarHalfYear,CalendarHalfYearLabel,CalendarQuarter,CalendarQuarterLabel,CalendarMonth,CalendarMonthLabel,CalendarWeek,CalendarWeekLabel,CalendarDayOfWeek,CalendarDayOfWeekLabel,FiscalYear,FiscalYearLabel,FiscalHalfYear,FiscalHalfYearLabel,FiscalQuarter,FiscalQuarterLabel,FiscalMonth,FiscalMonthLabel,IsWorkDay,IsHoliday,EuropeSeason,NorthAmericaSeason,AsiaSeason
0,2005-01-01,2005,Year 2005,20051,H1,20051,Q1,200501,January,200501,Week 1,2005017,Saturday,2005,FiscalYear 2005,20051,H1,20051,Q1,200501,Month 1,WeekEnd,0,Holiday,Spring/Back to Business,Holiday
1,2005-01-02,2005,Year 2005,20051,H1,20051,Q1,200501,January,200502,Week 2,2005021,Sunday,2005,FiscalYear 2005,20051,H1,20051,Q1,200501,Month 1,WeekEnd,0,Holiday,Spring/Back to Business,Holiday
2,2005-01-03,2005,Year 2005,20051,H1,20051,Q1,200501,January,200502,Week 2,2005022,Monday,2005,FiscalYear 2005,20051,H1,20051,Q1,200501,Month 1,WorkDay,0,Holiday,Spring/Back to Business,Holiday
3,2005-01-04,2005,Year 2005,20051,H1,20051,Q1,200501,January,200502,Week 2,2005023,Tuesday,2005,FiscalYear 2005,20051,H1,20051,Q1,200501,Month 1,WorkDay,0,Holiday,Spring/Back to Business,Holiday
4,2005-01-05,2005,Year 2005,20051,H1,20051,Q1,200501,January,200502,Week 2,2005024,Wednesday,2005,FiscalYear 2005,20051,H1,20051,Q1,200501,Month 1,WorkDay,0,Holiday,Spring/Back to Business,Holiday


## Handling missing value

In [4]:
dim_date.isna().sum()

Datekey                   0
CalendarYear              0
CalendarYearLabel         0
CalendarHalfYear          0
CalendarHalfYearLabel     0
CalendarQuarter           0
CalendarQuarterLabel      0
CalendarMonth             0
CalendarMonthLabel        0
CalendarWeek              0
CalendarWeekLabel         0
CalendarDayOfWeek         0
CalendarDayOfWeekLabel    0
FiscalYear                0
FiscalYearLabel           0
FiscalHalfYear            0
FiscalHalfYearLabel       0
FiscalQuarter             0
FiscalQuarterLabel        0
FiscalMonth               0
FiscalMonthLabel          0
IsWorkDay                 0
IsHoliday                 0
EuropeSeason              0
NorthAmericaSeason        0
AsiaSeason                0
dtype: int64

In [5]:
dim_date['EuropeSeason'].unique()

array(['Holiday', 'Spring/Business', 'None', 'Back to School'],
      dtype=object)

In [6]:
dim_date.loc[dim_date['EuropeSeason'] == 'None', 'EuropeSeason'] ='No Season'

In [7]:
dim_date['NorthAmericaSeason'].unique()

array(['Spring/Back to Business', 'None', 'Back to School', 'Holiday'],
      dtype=object)

In [8]:
dim_date.loc[dim_date['NorthAmericaSeason'] == 'None', 'NorthAmericaSeason'] ='No Season'

In [9]:
dim_date['AsiaSeason'].unique()

array(['Holiday', 'Spring/Back to School', 'Tax Time/Summer', 'None'],
      dtype=object)

In [10]:
dim_date.loc[dim_date['AsiaSeason'] == 'None', 'AsiaSeason'] ='No Season'

**All Nulls cleared.**

## Datatype check

In [11]:
dim_date.dtypes

Datekey                   datetime64[ns]
CalendarYear                       int64
CalendarYearLabel                 object
CalendarHalfYear                   int64
CalendarHalfYearLabel             object
CalendarQuarter                    int64
CalendarQuarterLabel              object
CalendarMonth                      int64
CalendarMonthLabel                object
CalendarWeek                       int64
CalendarWeekLabel                 object
CalendarDayOfWeek                  int64
CalendarDayOfWeekLabel            object
FiscalYear                         int64
FiscalYearLabel                   object
FiscalHalfYear                     int64
FiscalHalfYearLabel               object
FiscalQuarter                      int64
FiscalQuarterLabel                object
FiscalMonth                        int64
FiscalMonthLabel                  object
IsWorkDay                         object
IsHoliday                          int64
EuropeSeason                      object
NorthAmericaSeas

In [12]:
dim_date['Datekey'] = pd.to_datetime(dim_date['Datekey'])

In [13]:
dim_date['IsHoliday'].unique()

array([0], dtype=int64)

Issue: All values is 0 

In [14]:
dim_date['IsWorkDay'].unique()

array(['WeekEnd', 'WorkDay'], dtype=object)

In [15]:
dim_date.groupby('IsWorkDay')['IsWorkDay'].count()

IsWorkDay
WeekEnd     731
WorkDay    1825
Name: IsWorkDay, dtype: int64

In [16]:
dim_date.loc[dim_date['IsWorkDay'] == 'WeekEnd', 'IsHoliday'] = 1

In [17]:
dim_date.groupby('IsHoliday')['IsHoliday'].count()

IsHoliday
0    1825
1     731
Name: IsHoliday, dtype: int64

## Standardization 

In [18]:
#rename `Datekey to DateKey
dim_date.rename(columns ={'Datekey': 'DateKey'}, inplace =True) 

In [19]:
# Retrieve month from datetime
dim_date['MonthNumber'] =  dim_date['DateKey'].dt.month

dim_date['CalendarDayOfWeekNumber'] = dim_date['DateKey'].dt.dayofweek

In [20]:
dim_date = dim_date.drop(['FiscalHalfYear', 'FiscalMonth', 'CalendarQuarter', 'CalendarHalfYear', 'CalendarMonth', 'CalendarWeek', 'CalendarDayOfWeek', 'FiscalQuarter'], axis=1)

In [21]:
dim_date.head()

,DateKey,CalendarYear,CalendarYearLabel,CalendarHalfYearLabel,CalendarQuarterLabel,CalendarMonthLabel,CalendarWeekLabel,CalendarDayOfWeekLabel,FiscalYear,FiscalYearLabel,FiscalHalfYearLabel,FiscalQuarterLabel,FiscalMonthLabel,IsWorkDay,IsHoliday,EuropeSeason,NorthAmericaSeason,AsiaSeason,MonthNumber,CalendarDayOfWeekNumber
0,2005-01-01,2005,Year 2005,H1,Q1,January,Week 1,Saturday,2005,FiscalYear 2005,H1,Q1,Month 1,WeekEnd,1,Holiday,Spring/Back to Business,Holiday,1,5
1,2005-01-02,2005,Year 2005,H1,Q1,January,Week 2,Sunday,2005,FiscalYear 2005,H1,Q1,Month 1,WeekEnd,1,Holiday,Spring/Back to Business,Holiday,1,6
2,2005-01-03,2005,Year 2005,H1,Q1,January,Week 2,Monday,2005,FiscalYear 2005,H1,Q1,Month 1,WorkDay,0,Holiday,Spring/Back to Business,Holiday,1,0
3,2005-01-04,2005,Year 2005,H1,Q1,January,Week 2,Tuesday,2005,FiscalYear 2005,H1,Q1,Month 1,WorkDay,0,Holiday,Spring/Back to Business,Holiday,1,1
4,2005-01-05,2005,Year 2005,H1,Q1,January,Week 2,Wednesday,2005,FiscalYear 2005,H1,Q1,Month 1,WorkDay,0,Holiday,Spring/Back to Business,Holiday,1,2


In [22]:
dim_date.dtypes

DateKey                    datetime64[ns]
CalendarYear                        int64
CalendarYearLabel                  object
CalendarHalfYearLabel              object
CalendarQuarterLabel               object
CalendarMonthLabel                 object
CalendarWeekLabel                  object
CalendarDayOfWeekLabel             object
FiscalYear                          int64
FiscalYearLabel                    object
FiscalHalfYearLabel                object
FiscalQuarterLabel                 object
FiscalMonthLabel                   object
IsWorkDay                          object
IsHoliday                           int64
EuropeSeason                       object
NorthAmericaSeason                 object
AsiaSeason                         object
MonthNumber                         int32
CalendarDayOfWeekNumber             int32
dtype: object

In [23]:
engine = create_engine(f'mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes')

dim_date.to_sql('DimDate_temp', engine, if_exists='replace', index=False)

60

Error: ('23000', '[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The DELETE statement conflicted with the REFERENCE constraint "FK_FactExchangeRate_DimDate". The conflict occurred in database "ContosoRetailDW", table "dbo.FactExchangeRate", column \'DateKey\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)')
